In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
data_dir = os.path.join(os.getcwd(), 'simulation_results')

##### 1.

In [3]:
scenarios = ['clear-night', 'clear-noon', 'clear-sunset', 'haze-noon', 'haze-sunset', 'rain-noon']
csvs = ['ctl', 'cvip', 'traj']

In [4]:
all_data = []
dfs = {}
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for scenario in scenarios:
            if scenario in folder:
                folder = scenario
                break

        for file in os.listdir(folder_path):
            if file.endswith('.csv'):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                for csv_type in csvs:
                    if csv_type in file:
                        file = csv_type
                        break
                all_data.append((folder, file, df))
                key = f"{folder}_{file}"
                dfs[key] = df

for i, (folder, file, df) in enumerate(all_data):
    print(f"Data from folder '{folder}', file '{file}':\n", df.head(), "\n")
    print("="*50)

Data from folder 'clear-night', file 'ctl':
        ts  agent_id  throttle     steer  brake
0  542558         0       0.9 -0.003379    0.0
1  542559         0       0.9 -0.010903    0.0
2  542560         0       0.9 -0.009867    0.0
3  542561         0       0.9 -0.011255    0.0
4  542562         0       0.9  0.000766    0.0 

Data from folder 'clear-night', file 'cvip':
        ts  agent_id        cvip      cvip_x     cvip_y      cvip_z
0  542558         0  500.491189  198.767441 -95.832657 -499.819366
1  542559         0    5.595580  195.567444 -90.832657    0.100000
2  542560         0    5.592365  195.567444 -90.832657    0.095407
3  542561         0    5.589578  195.567444 -90.832657    0.084691
4  542562         0    5.587154  195.567444 -90.832657    0.069311 

Data from folder 'clear-night', file 'traj':
        ts  agent_id           x         y         z    v
0  542558         0  192.362411 -86.26268  0.539326  0.0
1  542559         0  192.362411 -86.26268  0.491906  0.0
2  5

In [ ]:
# List the first 5 rows of the ctl.csv, cvip.csv, and traj.csv with the clear-night weather condition.
scenario = 'clear-night'

for file_type in csvs:
    key = f"{scenario}_{file_type}" 
    df = dfs.get(key)  
    if df is not None:
        # print(df['agent_id'].nunique())
        print(f"Data from '{key}':\n", df.head())

##### 2.

In [ ]:
# The duration of the scene
duration = 0
for scenario in scenarios:
    for file_type in csvs:
        key = f"{scenario}_{file_type}" 
        df = dfs.get(key)  
        if df is not None:
            start_time = df['ts'].min() 
            end_time = df['ts'].max()  
            duration = end_time - start_time

    print(f"Scene '{scenario}'")
    print(f"The duration of the scene: {duration} seconds")
    print("="*50)

In [ ]:
# Mean and standard deviation of the values of the features
features = ['throttle', 'steer', 'brake', 'cvip', 'x', 'y', 'v']
results_df = pd.DataFrame(index=features)

for scenario in scenarios:
    mean_std_values = {}
    for file_type in csvs:
        key = f"{scenario}_{file_type}" 
        df = dfs.get(key) 
        if df is not None:
            for feature in features:
                if feature in df.columns:
                    mean_value = df[feature].mean()
                    std_value = df[feature].std()
                    mean_std_values[feature] = f"{mean_value:.3f} ({std_value:.3f})"
    
    if mean_std_values:
        results_df[scenario] = pd.Series(mean_std_values)

results_df

In [ ]:
# colors, matching with scenarios
colors = ['tomato', 'crimson', 'darkorange', 'cornflowerblue', 'lightseagreen', 'lightslategrey']

for feature in features:
    plt.figure(figsize=(10, 6)) 
    plt.title(f"{feature.capitalize()} vs Time for different weather conditions")
    plt.xlabel("Time (ts)")
    plt.ylabel(f"{feature.capitalize()} value")

    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None 
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key) 
            if df is not None and feature in df.columns:
                df['ts'] = pd.to_datetime(df['ts'], unit='s')
                plt.plot(df['ts'], df[feature], label=scenario, color=color, alpha=0.7)

    # second
    plt.gca().xaxis.set_major_locator(mdates.SecondLocator(interval=1000))  
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))  

    plt.xticks(rotation=45)
    plt.legend(title="Weather Conditions", loc='lower right')
    plt.tight_layout()  
    plt.show()

##### task1.3 plot time as an ordinal variable

In [ ]:
# pile
for feature in features:
    plt.figure(figsize=(10, 6)) 
    plt.title(f"{feature.capitalize()} vs Time for different weather conditions")
    plt.xlabel("Index")
    plt.ylabel(f"{feature.capitalize()} value")

    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None 
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key) 
            if df is not None and feature in df.columns:
                plt.plot(df.index, df[feature], label=scenario, color=color, alpha=0.7)

    plt.xticks(rotation=45) 
    plt.legend(title="Weather Conditions", loc='lower right')
    plt.tight_layout() 
    plt.show()

##### task1.3 subplot

In [ ]:
for feature in features:
    fig, ax = plt.subplots(len(scenarios), 1, figsize=(10, 6 * len(scenarios)))

    if len(scenarios) == 1:
        ax = [ax]

    for i, (scenario, color) in enumerate(zip(scenarios, colors)):
        df = None  
        for file_type in csvs:
            key = f"{scenario}_{file_type}"  
            df = dfs.get(key)  
            if df is not None and feature in df.columns:
                ax[i].plot(df['ts'], df[feature], label=scenario, color=color, alpha=0.7)
                ax[i].set_title(f"{feature.capitalize()} vs Time for {scenario}")
                ax[i].set_xlabel("Time (ts)")
                ax[i].set_ylabel(f"{feature.capitalize()} value")
                ax[i].legend(loc='lower right') 
    plt.tight_layout()
    plt.show()